<p style="text-align: center;font-size: 40pt">Roslaunch</p>
<p style="text-align: center;font-size: 20pt">A tool for easily launching multiple ROS nodes</p>
<img src="images/roslaunch_title.png" width="100%">

So far, we have mostly launched ROS nodes by using the `rosrun` command, which runs a single node.
In more complex robotic systems, there will be several nodes running in parallel.
It would be impractical to have a new terminal session started for each of them.
Also, even launching such a system would be a demanding process.

ROS offers a solution: the `roslaunch` tool.
Its purpose is to launch groups of ROS nodes automatically based on XML text files that list the necessary nodes, their parameters and possibly includes other launch files as well.
While a single node would be run by calling `rosrun [package_name] [node_executable_name]`, the corresponding call for starting a launch file is `roslaunch [package_name] [launch_file_name]`.

For example, when you were playing with the turtle simulator in the introduction to ROS, you were running a single node by executing `rosrun turtlesim turtlesim_node`.
On the other hand, the more complicated tutorial on TF was launching the two chasing turtles by running `roslaunch turtle_tf turtle_tf_demo.launch`.
That launch file actually starts five different ROS nodes which work together to read the keyboard commands, display the graphical window with turtles and manage the computations necessary for the chasing turtle to follow the manually controlled turtle.
You can see the actual contents of that launch file in the image above.
The same output could be achieved by by separately running all the nodes using the `rosrun` command, but this way is so much more convenient...


# ROS Parameter Server

In the introductory lesson to ROS, we have skipped the [Parameter Server](http://wiki.ros.org/Parameter%20Server) functionality offered by the _roscore_.
We will catch-up now by looking at the ROS tutorial on this topic.
It will help us understand the contents of ROS launch files, since the ROS parameters are often used there to configure the launched ROS nodes.

The concept is explained in detail on the [Parameter Server](http://wiki.ros.org/Parameter%20Server) page, but it is really simple.
The _roscore_ process can store parameters, which are accessible by all ROS nodes and also manually from a terminal.
Each parameter has a name and a value.
The value can be a number, a piece of text (string), logical value (True or False) and some other types.
We can set them, read them, and clear them.
If you terminate the _roscore_, you loose the parameters set there.
We have already touched this mechanism before when playing with our small bag file from the DARPA competition.
You have executed the `rosparam set use_sim_time true` command to set the `use_sim_time` parameter to the `true` value telling all the running ROS nodes that we were running with a simulated time.

Now, to familiarize more with the terminal commands related to the ROS parameters, proceed to the [Understanding ROS Services and Parameters](http://wiki.ros.org/ROS/Tutorials/UnderstandingServicesParams) and focus on Section 3, which shows you how to set and read ROS parameters from the parameter server.
They let you play with parameters that configure the turtle simulator - you need one running, you can start it as usual by calling `rosrun turtlesim turtlesim_node`.

# How to compose and understand a launch file

Ok, now we understand what ROS parameters are.
That is good because the launch files are usually full of them, besides the ROS nodes and a few other features.

The syntax of a launch file follows the XML standard.
Let's dissect the launch file from the TF tutorial you might have already used.
You can open the launch file by going into the directory which contains them: `roscd turtle_tf/launch` and then opening the file with
```
$ gedit turtle_tf_demo.launch 
```
The executable `gedit` is a simple text editor installed by default on Ubuntu.

You can see right away that the contents of the launch file are between tags that looks like this:
```xml
<launch>
 
    ...
  
</launch>
```

Now what can we put between them?

## The \<node\> tag
    
A new node to be launched by the launch file is defined by the 
```xml
<node name="..." pkg="..." type="..." respawn="..." output="..." >
    ...
</node>
```
pair of tags, or if you have nothing to add between them, just by
```xml
<node name="..." pkg="..." type="..." respawn="..." output="..." />
```
using a single tag.
The attributes of the _node_ tag are documented in Section 1.2 of the [node tag documentation](http://wiki.ros.org/roslaunch/XML/node) page.
The most important, non-optional attributes are `name`, `pkg` and `type`.
The _name_ attribute sets the name for the node which must be unique for the whole ROS network running under one _roscore_.
This name is the name you see in `rosnode list` or in the _rqt_graph_ tool.
The _pkg_ is the name of the ROS package that contains the node executable.
The _type_ is the actual executable to be launched to become a running ROS node.
It is usually a C++ compiled binary or a Python script.

## The \<param\> tag

This one is less complicated, its mandatory attributes are just _name_ and _value_ and they correspond to what you already know about ROS parameters.
The tag itself can be placed to the highest level in the launch file between the `<launch> ... </launch>` tags, such as:
```xml
<launch>

  ...

  <param name="scale_linear" value="2" type="double"/>
  
  ...
  
</launch>
```
as taken from the title image above. Then there will be a `/scale_angular` parameter in the parameter server, which you can read by calling `rosparam get /scale_angular`.
Or, the _param_ tag can be placed between _node_ tags, the parameter is then placed into a namespace according to the _node_ name.
To demonstrate this, let's have a look at the other part of the example:
```xml
<launch>

  ...
  
  <node name="turtle1_tf_broadcaster" pkg="turtle_tf" type="turtle_tf_broadcaster.py" respawn="false" output="screen" >
    <param name="turtle" type="string" value="turtle1" />
  </node>
  
  ...
  
</launch>
```
In this case, the parameter is located in the namespace of the "_turtle1_tf_broadcaster_" node, so in the parameter server, you will find its value by calling `rosparam get /turtle1_tf_broadcaster/turtle`. 

## The \<remap\> tag

The last tag we present in this lesson is the `remap` tag.
It allows us to change names of resources in ROS from within a launch file.
This is a powerful feature in ROS which allows flexible re-configuration of its components.

Imagine you have a node, which by default publishes images on a topic called "_output_".
For your purposes, you want to change the name to "_camera_out_".
Then all that needs to be done is adding the _remap_ tag:
```xml
<launch>

  ...
  
  <node name="[node name]" pkg="[package name]" type="[executable name]">
    <remap from="output" to="camera_out"/>
  </node>
  
  ...
  
</launch>
```

# Wrap-up

Now you should be able to understand basic contents of launch files.

The following [exercise](../../exercises/2-ex_ros/2.5-exercises_roslaunch.ipynb) will let you write one by yourself.

After that, it will be time to prepare for your exam through this small [exercise](../../exercises/2-ex_ros/2.6-debugging_simple_node.ipynb):
- [Don't mess with tomatoes](2.6-exam_ros.ipynb)

&#8635; [Go back to the list of lessons](2.0-overview.ipynb)